In [1]:
import numpy as np
import networkx as nx
from itertools import product

In [30]:
#try tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Input

model = Sequential([
    Input(shape = (16,),name="inlayer"),
    Dense(5,activation ="relu",name="dense1"),
    Dense(5,activation ="relu",name="dense2"),
    Dense(2,name="classes")
])


x = np.random.randn(16*2*10).reshape((20,16))
y = np.random.randn(20*2).reshape((20,2))

model.compile(loss="mse",optimizer="adam")
model.fit(x,y,epochs=20,verbose = 0)

In [48]:
from tensorflow.keras import backend
from tensorflow.keras.models import Model

get_intermediate_layer = backend.function([model.layers[0].input],[model.layers[1].output])
int_layer = Model(inputs = model.layers[0].input, outputs = [model.get_layer("dense1").input,model.get_layer("dense2").input,model.get_layer("classes").input])
layer_last  =int_layer.predict(x)

layer_inputs = {"dense1":layer_last[0],"dense2":layer_last[1],"classes":layer_last[2]}

In [186]:
def naming(length,pre):
    return list(map(lambda x: pre+"_{}".format(x),list(range(length))))

def full_connect(graph,start,end):
    for i,j in product(start,end):
        graph.add_edge(i,j)
    start.extend(end)
    return graph.subgraph(start+end)

def add_weight_full(start,end,graph,weight):
    layer_weights = model.get_layer(start).get_weights()       
    layer_last  = int_layer.predict(x)
    for i in graph.edges:
        start_node = int(i[0].split("_")[-1])
        end_node = int(i[1].split("_")[-1])
        graph[i[0]][i[1]]["weight"] = layer_weights[0][end_node,start_node]*layer_last[0,end_node] - layer_weights[1][start_node]
        print(G[i[0]][i[1]])
    
def add_weight_to_edge(start,end,graph,model,layer_inputs):
    start_splitted = start.split("_")
    end_splitted = end.split("_")
    start_layer = start_splitted[0]
    start_node = int(start_splitted[1])
    end_node = int(end_splitted[1])
    layer = model.get_layer(start_layer)
    layer_weights = layer.weights
    layer_inputs_node = layer_inputs[start_layer]
    bias = layer_weights[1][end_node] / layer_weights[0].shape[1]
    bias = bias.numpy()
    edge = layer_weights[0][start_node,end_node] * layer_inputs_node[0][start_node] 
    edge = edge.numpy()
    print(edge)
    print(bias)    
    graph[end][start]["weight"] = 1/np.max([0.00001,edge - bias])    
    
k = 2
d2 = 5
d1 = 5

inlayer = 16

d2_layer = naming(d2,"dense2") 
d1_layer = naming(d1,"dense1")
k_layer = naming(k,"classes")
inlayer = naming(inlayer,"inlayer")

G = nx.DiGraph()
G.add_nodes_from(k_layer)
G.add_nodes_from(inlayer)
G.add_nodes_from(d2_layer)
G.add_nodes_from(d1_layer)

k_d2 = full_connect(G,k_layer,d2_layer)
d2_d1 = full_connect(G,d2_layer,d1_layer)
d1_in = full_connect(G,d1_layer,inlayer)
in_end = full_connect(G,inlayer,["end"])



#weights to network
#layer_weights = model.layers[-1].get_weights()
#layer_last  = int_layer.predict(x)
#for i in k_d2.edges:
#    start_node = int(i[0].split("_")[-1])

#    end_node = int(i[1].split("_")[-1])
#    G[i[0]][i[1]]["weight"] = layer_weights[0][end_node,start_node]*layer_last[0,end_node] - layer_weights[1][start_node]
#    print(G[i[0]][i[1]])


In [89]:
start = "dense1_0"
end = "dense2_0"
graph = G

start_splitted = start.split("_")
end_splitted = end.split("_")
start_layer = start_splitted[0]
start_node = int(start_splitted[1])
end_node = int(end_splitted[1])
layer = model.get_layer(start_layer)
layer_weights = layer.weights
layer_inputs_node = layer_inputs[start_layer]
bias = layer_weights[1][end_node] / layer_weights[0].shape[1]
bias = bias.numpy()
edge = layer_weights[0][start_node,end_node] * layer_inputs_node[0][start_node] 
edge = edge.numpy()
graph[end][start]["weight"] = edge - bias    

In [149]:
add_weight_to_edge(start,end,G,model,layer_inputs)

TypeError: 'numpy.float32' object cannot be interpreted as an integer

In [185]:
edge

0.15558237

In [187]:
#add weights to all edges
for i,j in G.edges:
    try:
        add_weight_to_edge(j,i,G,model,layer_inputs)
    except:
        print(i,j)
        continue

0.0
-0.0038814235
-0.0
-0.0038814235
0.28274533
-0.0038814235
-0.0
-0.0038814235
-0.0
-0.0038814235
0.0
-0.0034509513
-0.0
-0.0034509513
-0.10865741
-0.0034509513
-0.0
-0.0034509513
0.0
-0.0034509513
inlayer_0 end
inlayer_1 end
inlayer_2 end
inlayer_3 end
inlayer_4 end
inlayer_5 end
inlayer_6 end
inlayer_7 end
inlayer_8 end
inlayer_9 end
inlayer_10 end
inlayer_11 end
inlayer_12 end
inlayer_13 end
inlayer_14 end
inlayer_15 end
0.15558237
-0.0037132718
0.024921438
-0.0037132718
-0.14910682
-0.0037132718
0.21545057
-0.0037132718
-0.26844952
-0.0037132718
0.46381184
-0.0040353313
0.0126817925
-0.0040353313
-0.06635935
-0.0040353313
-0.15090765
-0.0040353313
0.27754745
-0.0040353313
-0.24894632
-0.0038513
-0.0041742376
-0.0038513
0.10606072
-0.0038513
-0.19108412
-0.0038513
-0.33665562
-0.0038513
0.3958807
-0.004010816
0.009016857
-0.004010816
-0.42719242
-0.004010816
-0.18452898
-0.004010816
0.07422716
-0.004010816
-0.48694026
-0.003940221
-0.03625872
-0.003940221
0.41352245
-0.003940221
0

In [188]:
for i,j in G.edges:
    print(i,j,G[i][j])

classes_0 dense2_0 {'weight': 257.6374362822377}
classes_0 dense2_1 {'weight': 257.6374362822377}
classes_0 dense2_2 {'weight': 3.488857820840505}
classes_0 dense2_3 {'weight': 257.6374362822377}
classes_0 dense2_4 {'weight': 257.6374362822377}
classes_1 dense2_0 {'weight': 289.7751695939804}
classes_1 dense2_1 {'weight': 289.7751695939804}
classes_1 dense2_2 {'weight': 99999.99999999999}
classes_1 dense2_3 {'weight': 289.7751695939804}
classes_1 dense2_4 {'weight': 289.7751695939804}
inlayer_0 end {}
inlayer_1 end {}
inlayer_2 end {}
inlayer_3 end {}
inlayer_4 end {}
inlayer_5 end {}
inlayer_6 end {}
inlayer_7 end {}
inlayer_8 end {}
inlayer_9 end {}
inlayer_10 end {}
inlayer_11 end {}
inlayer_12 end {}
inlayer_13 end {}
inlayer_14 end {}
inlayer_15 end {}
dense2_0 dense1_0 {'weight': 6.277635980564911}
dense2_0 dense1_1 {'weight': 34.92265127566952}
dense2_0 dense1_2 {'weight': 99999.99999999999}
dense2_0 dense1_3 {'weight': 4.562796593598134}
dense2_0 dense1_4 {'weight': 99999.99999

In [184]:
k_d2 = G.subgraph(d2_layer+d1_layer+k_layer)

In [131]:
for i,j in k_d2.edges:
    print(i,j,k_d2[i][j])

classes_0 dense2_0 {'weight': 0.0038814235}
classes_0 dense2_1 {'weight': 0.0038814235}
classes_0 dense2_2 {'weight': 0.28662676}
classes_0 dense2_3 {'weight': 0.0038814235}
classes_0 dense2_4 {'weight': 0.0038814235}
classes_1 dense2_0 {'weight': 0.0034509513}
classes_1 dense2_1 {'weight': 0.0034509513}
classes_1 dense2_2 {'weight': -0.10520646}
classes_1 dense2_3 {'weight': 0.0034509513}
classes_1 dense2_4 {'weight': 0.0034509513}
dense2_0 dense1_0 {'weight': 0.15929563}
dense2_0 dense1_1 {'weight': 0.02863471}
dense2_0 dense1_2 {'weight': -0.14539355}
dense2_0 dense1_3 {'weight': 0.21916384}
dense2_0 dense1_4 {'weight': -0.26473624}
dense2_1 dense1_0 {'weight': 0.46784717}
dense2_1 dense1_1 {'weight': 0.016717125}
dense2_1 dense1_2 {'weight': -0.062324017}
dense2_1 dense1_3 {'weight': -0.14687231}
dense2_1 dense1_4 {'weight': 0.28158277}
dense2_2 dense1_0 {'weight': -0.24509503}
dense2_2 dense1_1 {'weight': -0.0003229375}
dense2_2 dense1_2 {'weight': 0.10991202}
dense2_2 dense1_3 {'

In [133]:
k_d2.remove_nodes_from(inlayer)

NetworkXError: Frozen graph can't be modified

In [189]:
G.add_node("start")

In [190]:
full_connect(G,d1_layer,["start"])    

In [191]:
for i,j in G.edges:
    if j == "start":
        G[i][j]["weight"] = 1

In [192]:
G[i][j]["weight"]

1

In [193]:
full_connect(G,["end_node"],k_layer)

for i,j in G.edges:
    if i == "end_node":
        G[i][j]["weight"] = 1

In [194]:
nx.shortest_path(G,"end_node","start",weight="weight")

['end_node', 'dense2_1', 'dense1_0', 'start']

In [195]:
nx.shortest_path_length(G,"end_node","start",weight="weight")

4.137450149206802

In [203]:
paths = nx.all_shortest_paths(G,"end_node","start",weight="weight")

In [204]:
for i in paths:
    print(i)

['end_node', 'dense2_1', 'dense1_0', 'start']


In [213]:

for i in nx.shortest_simple_paths(G,"end_node","start",weight="weight"):
    w = 0
    for j in range(len(i)-1):
        w += G[i[j]][i[j+1]]["weight"]
    print(i,w)

['end_node', 'dense2_1', 'dense1_0', 'start'] 4.137450149206802
['end_node', 'dense2_4', 'dense1_2', 'start'] 4.395423724469708
['end_node', 'dense2_3', 'dense1_0', 'start'] 4.500678149486221


KeyError: 'weight'